In [90]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import common.common_machine_learning as common

#pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [91]:
def evaluar_rf(modelo, X_test, y_test):
    y_pred = modelo.predict(X_test)
    errors = abs(y_pred - y_test)
    mape = 100 * np.mean(errors / y_test)
    accuracy = 100 - mape
    print('Performance del modelo:')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [92]:
MAX_ITER = 10

## Limpieza de datasets

### Elijo columnas basándome en la importancia según Univariate

In [93]:
TARGET = 'precio'
columnas = ["tipodepropiedad", "provincia", "antiguedad", "metroscubiertos",
           "metrostotales", "gimnasio", "usosmultiples", "piscina", "escuelascercanas", "centroscomercialescercanos", "precio"]

In [94]:
train = pd.read_csv('sets_de_datos/train.csv', index_col = 0)
test = pd.read_csv('sets_de_datos/test.csv', index_col = 0)

### Limpio el set train: elimino columnas con nans, relleno con promedios, etc

In [95]:
train = train[columnas]

In [96]:
train.isna().sum()

tipodepropiedad                  46
provincia                       155
antiguedad                    43555
metroscubiertos               17400
metrostotales                 51467
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
precio                            0
dtype: int64

In [97]:
train["antiguedad"].fillna(train["antiguedad"].mean(),inplace=True)
train["metroscubiertos"].fillna(train["metroscubiertos"].mean(), inplace=True)
train["metrostotales"].fillna(train["metrostotales"].mean(), inplace=True)
train["precio"].fillna(train["precio"].mean(), inplace=True)
train.dropna(subset=["tipodepropiedad"], inplace=True)
train.fillna(0, inplace=True)

#filtro la que no están en test para que train y test tengan las mismas columnas tras el encoding
train["tipodepropiedad"] = train[train["tipodepropiedad"].isin(test["tipodepropiedad"].unique())]
train.dropna(subset=["tipodepropiedad"], inplace=True)
train['provincia'] = train['provincia'].str.replace("provincia_0","nan")


In [98]:
train.isna().sum()

tipodepropiedad                 0
provincia                     153
antiguedad                      0
metroscubiertos                 0
metrostotales                   0
gimnasio                        0
usosmultiples                   0
piscina                         0
escuelascercanas                0
centroscomercialescercanos      0
precio                          0
dtype: int64

### Limpio el set test: relleno con promedios y moda

In [99]:
columnas_test = ["tipodepropiedad", "provincia", "antiguedad", "metroscubiertos",
                 "metrostotales", "gimnasio", "usosmultiples", "piscina", "escuelascercanas", "centroscomercialescercanos"]
test = test[columnas_test]

In [100]:
test.isna().sum()

tipodepropiedad                   7
provincia                        42
antiguedad                    10714
metroscubiertos                4299
metrostotales                 12655
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
dtype: int64

In [101]:
#ciudad = test["ciudad"].mode()
#tipodepropiedad = test["tipodepropiedad"].mode()

test = test.fillna(value = {'tipodepropiedad' : test["tipodepropiedad"].mode().to_string(),
                            'provincia' : test["provincia"].mode().to_string()})
#test["tipodepropiedad"].fillna(test["ciudad"].mode(), inplace=True)
#est["ciudad"].fillna(test["ciudad"].mode(), inplace=True)
test["antiguedad"].fillna(test["antiguedad"].mean(), inplace=True)
test["metroscubiertos"].fillna(test["metroscubiertos"].mean(), inplace=True)
test["metrostotales"].fillna(test["metrostotales"].mean(), inplace=True)

test['provincia'] = test['provincia'].str.replace('0    Distrito Federal','Distrito Federal') #Sino, explota el universo >:[
test['tipodepropiedad'] = test['tipodepropiedad'].str.replace('0    Casa',"Casa")

In [102]:
test.isna().sum()

tipodepropiedad               0
provincia                     0
antiguedad                    0
metroscubiertos               0
metrostotales                 0
gimnasio                      0
usosmultiples                 0
piscina                       0
escuelascercanas              0
centroscomercialescercanos    0
dtype: int64

# Entrenamiento

### One Hot Encoding para train y test

In [103]:
train_OHE  = pd.get_dummies(train)

In [104]:
test_OHE = pd.get_dummies(test)

### Divido train

In [105]:
X = train_OHE.drop([TARGET], axis = 1).copy().values
y = list(train_OHE[TARGET].copy())

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)

### Hiperparámetros a tunear

In [107]:
hiperparametros = {
    "n_estimators": [100, 200, 250 ,300, 350 ,400, 500],
    "max_depth": [10, 20, 30, 40, 50, 60],
    "max_features": [1, 5, 'auto', 'sqrt', 'log2'],
    "min_samples_leaf": [1, 2, 4],
    "min_samples_split": [2, 5, 10],
}

### Tuneo

In [108]:
rf = RandomForestRegressor()

inicio = time.time()


rf_random = RandomizedSearchCV(estimator = rf, param_distributions = hiperparametros, n_iter = MAX_ITER, cv = 3, verbose=2, random_state=42, n_jobs = -1)

fin = time.time()

rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 30.9min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators='warn',
                                                   n_jobs=None, oob_score=False,
                                                   random_sta...s

In [109]:
print("El tuneo tardó: {} minutos.".format((fin - inicio) / 60))

El tuneo tardó: 0.0 minutos.


In [110]:
print("Mejores parámetros:")
print(rf_random.best_params_)

Mejores parámetros:
{'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 40}


### Entrenamiento

In [111]:
mejores_hiperparametros = rf_random.best_params_

In [112]:
mejor_rf = RandomForestRegressor(n_estimators = mejores_hiperparametros["n_estimators"],
                                 min_samples_split = mejores_hiperparametros["min_samples_split"],
                                 min_samples_leaf = mejores_hiperparametros["min_samples_leaf"],
                                 max_features = mejores_hiperparametros["max_features"],
                                 max_depth = mejores_hiperparametros["max_depth"])
mejor_rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=40,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=300,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [113]:
evaluar_rf(mejor_rf, X_test, y_test)

Performance del modelo:
Average Error: 730152.3149 degrees.
Accuracy = 64.68%.


64.67709726761025